In [1]:
from __future__ import print_function
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf
from tensorflow.python.ops import rnn_cell, rnn

In [2]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS_alone.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset= save['train_dataset']
    validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    test_old = save['test_dataset_evan']
    train_old = save['train_dataset_evan']
    del save


test_data = test_dataset['data']
test_label = test_dataset['label']
train_old_data = train_old['data']
train_old_label = train_old['label']

In [3]:
train_data = train_old_data
train_label =train_old_label

In [5]:
#dataset normalize
mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
print(mean.shape, std.shape)
train_data_n = (train_old_data - mean)/std
#validate_data_n = (validate_data - mean)/std
test_data_n = (test_data - mean)/std

(70,) (70,)


In [6]:
train_data.shape, test_data.shape

((202270, 70), (50569, 70))

In [7]:
train_data = np.hstack((train_data_n[:, :63], train_data_n[:, -1:],train_data_n[:, 63:-1] ))
test_data = np.hstack((test_data_n[:, :63], test_data_n[:, -1:],test_data_n[:, 63:-1] ))

In [8]:
train_data_2d = train_data.reshape(train_data.shape[0], 7, 10, 1)
test_data_2d = test_data.reshape(test_data.shape[0], 7, 10, 1)
train_label = train_label.reshape(-1, 1)
test_label = test_label.reshape(-1, 1)

In [9]:
train_data_2d.shape, train_label.shape

((202270, 7, 10, 1), (202270, 1))

regression problems

In [10]:
batch_size = 128
patch_size = 3
depth = 16
num_hidden = 64
num_channels = 1
num_labels = 1

In [16]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 7, 10, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_data_2d, dtype=tf.float32)
    tf_test_labels = tf.constant(test_label, dtype=tf.float32) 
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([10 * 7 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    pred = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.abs(pred - tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
    MAE = tf.reduce_mean(tf.abs(model(tf_test_dataset) - tf_test_labels))
    

In [18]:
num_steps = 100001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        batch_data = train_data_2d[offset:(offset + batch_size), :, :, :]
        batch_labels = train_label[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        if step % 50 == 0:
            print('Minibatch loss at step %d: loss = %f, MAE = %f' % (step, l, MAE.eval()))
            
    

Initialized
Minibatch loss at step 0: loss = 9.869551, MAE = 8.929339
Minibatch loss at step 50: loss = 0.535366, MAE = 1.221100
Minibatch loss at step 100: loss = 1.207199, MAE = 1.022987
Minibatch loss at step 150: loss = 0.482334, MAE = 0.996040
Minibatch loss at step 200: loss = 0.975134, MAE = 0.964097
Minibatch loss at step 250: loss = 0.448894, MAE = 1.119965
Minibatch loss at step 300: loss = 4.919765, MAE = 1.440712
Minibatch loss at step 350: loss = 4.172833, MAE = 1.212258
Minibatch loss at step 400: loss = 0.263186, MAE = 0.992542
Minibatch loss at step 450: loss = 0.711663, MAE = 0.932603
Minibatch loss at step 500: loss = 1.962352, MAE = 1.658382
Minibatch loss at step 550: loss = 0.429746, MAE = 0.971545
Minibatch loss at step 600: loss = 0.250949, MAE = 1.268905
Minibatch loss at step 650: loss = 0.744387, MAE = 0.996609
Minibatch loss at step 700: loss = 2.299270, MAE = 1.252802
Minibatch loss at step 750: loss = 0.321065, MAE = 0.936761
Minibatch loss at step 800: los

Classification problems

In [44]:
from __future__ import division

In [19]:
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(train_label.reshape(1, -1))
b_test_label = pre.transform(test_label.reshape(1, -1))

In [20]:
c_train_label = 1 - b_train_label[0]
c_test_label = 1 - b_test_label[0]

In [23]:
c_train_label[:10]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [24]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [25]:
v_train_label = makeIndicatorVars(c_train_label.reshape(-1, 1))
v_test_label = makeIndicatorVars(c_test_label.reshape(-1, 1))

In [27]:
v_train_label.shape, v_test_label.shape

((202270, 2), (50569, 2))

In [28]:
batch_size = 128
patch_size = 3
depth = 16
num_hidden = 64
num_channels = 1
num_labels = 2

In [48]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 7, 10, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_data_2d, dtype=tf.float32)
    tf_test_labels = tf.constant(v_test_label, dtype=tf.float32) 
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([10 * 7 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    def acc(predict, label):
        #correct_prediction = tf.equal(predicted_label, tf_train_label)
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        predict_event = tf.reduce_sum(tf.argmax(predict, 1))
        label_event = tf.reduce_sum(tf.argmax(label, 1))
        true_positive = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 2), tf.int64))
        true_negative = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 0), tf.int64))
        false_positive = predict_event - true_positive 
        false_negative = label_event - true_positive
        return accuracy, false_positive, false_negative, true_positive, true_negative
    def ROC(FP, FN, TP, TN):
        TP_percent = TP / (TP + FN) 
        FP_percent = FP / (FP + TN) 
        return TP_percent, FP_percent
    
    def PRC(FP, FN, TP, TN):
        precision = TP / (TP + FP + 1)
        recall = TP / (TP + FN + 1)
        f_score = 2 * precision * recall / (precision + recall)
        return precision, recall, f_score    
    
    # Training computation.
    pred = model(tf_train_dataset)
    #loss = tf.reduce_mean(tf.abs(pred - tf_train_labels))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, tf_train_labels)) # Softmax loss
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    
    # Evaluate model
    test_pred = model(tf_test_dataset)
    correct_pred = tf.equal(tf.argmax(test_pred,1), tf.argmax(tf_test_labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    test_acc, FP, FN, TP, TN = acc(test_pred, tf_test_labels)
    pre, rec, f_s = PRC(FP, FN, TP, TN)

In [49]:
num_steps = 100001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        batch_data = train_data_2d[offset:(offset + batch_size), :, :, :]
        batch_labels = v_train_label[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        if step % 50 == 0:
            print('Minibatch loss at step %d: loss = %f, MAE = %f' % (step, l, test_acc.eval()))
            tp, fp, fn, tn, precision, recall, f_score= session.run([TP, FP, FN, TN, pre, rec, f_s])
            print("TP = %d, FP = %d, FN = %d, TN = %d" % (tp, fp, fn, tn))
            print("precision = %f, recall = %f, f_score = %f" % (precision, recall, f_score))

Initialized
Minibatch loss at step 0: loss = 0.143919, MAE = 0.985722
TP = 0, FP = 0, FN = 722, TN = 49847
precision = 0.000000, recall = 0.000000, f_score = nan
Minibatch loss at step 50: loss = 0.002755, MAE = 0.985722
TP = 0, FP = 0, FN = 722, TN = 49847
precision = 0.000000, recall = 0.000000, f_score = nan
Minibatch loss at step 100: loss = 0.181885, MAE = 0.985722
TP = 0, FP = 0, FN = 722, TN = 49847
precision = 0.000000, recall = 0.000000, f_score = nan
Minibatch loss at step 150: loss = 0.005808, MAE = 0.985722
TP = 0, FP = 0, FN = 722, TN = 49847
precision = 0.000000, recall = 0.000000, f_score = nan
Minibatch loss at step 200: loss = 0.001898, MAE = 0.985722
TP = 0, FP = 0, FN = 722, TN = 49847
precision = 0.000000, recall = 0.000000, f_score = nan
Minibatch loss at step 250: loss = 0.003058, MAE = 0.985722
TP = 0, FP = 0, FN = 722, TN = 49847
precision = 0.000000, recall = 0.000000, f_score = nan
Minibatch loss at step 300: loss = 1.629452, MAE = 0.947616
TP = 466, FP = 2393